<a href="https://colab.research.google.com/github/DU-ds/pyspark_udemy/blob/main/Section3_8_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
! apt update
! apt install openjdk-8-jdk-headless -qq > /dev/null
! wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
! tar xf spark-2.3.1-bin-hadoop2.7.tgz
! pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

! ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark

from pyspark.sql import types
schema = types.StructType([
                           types.StructField('id', types.IntegerType()),
                           types.StructField("first_name", types.StringType()),
                           types.StructField("last_name", types.StringType()),
                           types.StructField("gender", types.StringType()),
                           types.StructField("City", types.StringType()),
                           types.StructField("JobTitle", types.StringType()),
                           types.StructField("Salary", types.StringType()),
                           types.StructField("Latitude", types.FloatType()),
                           types.StructField("Longitude", types.FloatType())
])

df = spark.read.csv("original.csv", header=True, schema=schema)
df.show()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [7]:
df = df.withColumn("clean_salary", df.Salary.substr(2, 100).cast("float"))
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|clean_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|    57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|     62846.6|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|    57576.52|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|    61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|    63863.09|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil En

In [8]:
df.groupBy("gender").agg(sqlf.sum('clean_salary').alias("total"),
                         sqlf.avg("clean_salary").alias("average"),
                         sqlf.min("clean_salary").alias("min"),
                         sqlf.max("clean_salary").alias("max")
                         ).show()

+------+--------------------+-----------------+--------+--------+
|gender|               total|          average|     min|     max|
+------+--------------------+-----------------+--------+--------+
|Female|2.7364519950195312E7|55618.94298820185|10616.44|99948.28|
|  Male|2.8123435678710938E7|55361.09385573019|10101.92|99942.92|
+------+--------------------+-----------------+--------+--------+



In [9]:

df.groupBy("gender", "city").agg(sqlf.sum('clean_salary').alias("total"),
                         sqlf.avg("clean_salary").alias("average"),
                         sqlf.min("clean_salary").alias("min"),
                         sqlf.max("clean_salary").alias("max")
                         ).show()

+------+-----------------+----------------+----------------+--------+--------+
|gender|             city|           total|         average|     min|     max|
+------+-----------------+----------------+----------------+--------+--------+
|Female|           Dachun| 25090.869140625| 25090.869140625|25090.87|25090.87|
|Female|      Trollhättan|106623.369140625|53311.6845703125|26830.47| 79792.9|
|  Male|          Wenshao| 18941.509765625| 18941.509765625|18941.51|18941.51|
|Female|            Lanas| 13765.900390625| 13765.900390625| 13765.9| 13765.9|
|  Male|            Mörön|    77940.078125|    77940.078125|77940.08|77940.08|
|Female|             Same|   73369.7265625|   73369.7265625|73369.73|73369.73|
|Female|          Sawahan|  24608.83984375|  24608.83984375|24608.84|24608.84|
|  Male|Monte da Boavista|     98586.71875|     98586.71875|98586.72|98586.72|
|Female|         Nusajaya|    71637.921875|    71637.921875|71637.92|71637.92|
|Female|            Kista|   96192.3984375|   96192.

In [15]:
lst = ["gender", "city", "JobTitle"]
df1 = df.groupBy(*lst).agg(sqlf.sum('clean_salary').alias("total"),
                         sqlf.avg("clean_salary").alias("average"),
                         sqlf.min("clean_salary").alias("min"),
                         sqlf.max("clean_salary").alias("max")
                         )
df1.show()

+------+--------------------+--------------------+----------------+----------------+--------+--------+
|gender|                city|            JobTitle|           total|         average|     min|     max|
+------+--------------------+--------------------+----------------+----------------+--------+--------+
|  Male|              Xijiao|         Developer I|13708.0302734375|13708.0302734375|13708.03|13708.03|
|Female|             Macarse|  Help Desk Operator|   68709.0234375|   68709.0234375|68709.02|68709.02|
|Female|          El Salitre|Systems Administr...|    94859.078125|    94859.078125|94859.08|94859.08|
|  Male|Santo Antônio do ...|        Food Chemist|        17283.25|        17283.25|17283.25|17283.25|
|  Male|                Bile|               Nurse|   97909.8828125|   97909.8828125|97909.88|97909.88|
|Female|            Welisara|     Legal Assistant|    48376.421875|    48376.421875|48376.42|48376.42|
|Female|              Dachun|                null| 25090.869140625| 25090

In [17]:
df1.write.csv("df1.csv")
df1.write.json("df1.json")
df1.write.parquet("df1.parquet")


AnalysisException: ignored

In [20]:
from google.colab import files
files.download("df1.csv")
files.download("df1.parquet")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
! cp -r "df1.csv" "/content/drive/My Drive/"
! cp -r "df1.parquet" "/content/drive/My Drive/"